In [2]:
print('this is mehedi')

this is mehedi


In [1]:
# importing os and load_dotenv
import os
from dotenv import load_dotenv

In [5]:
# loading dotenv and getting the openai api key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
# getting the openai key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [12]:
# importing PyPDFLoader from langchain.document_loaders
from langchain.document_loaders import PyPDFLoader

In [8]:
# showing the present working directory
%pwd

'd:\\genai_project\\Interview-Questions-Generator\\research'

In [ ]:
# gettingone step back from the present working directory
%cd ..

In [10]:
# showing the present working directory and we are one step back
%pwd

'd:\\genai_project\\Interview-Questions-Generator'

In [19]:
# path of the file daved into a variable
# calling the PyPDFLoader() class and creating an object
# and through the object call a function call functioned as load()
file_path = 'data/SDG.pdf'
loader = PyPDFLoader(file_path)
data = loader.load()

In [ ]:
# showing the whole dataset
data
# showing the data through indexing
data[0]
# length of the dataset
len(data)
# printint the dataset through length wise
for i in data:
    print(i.page_content)

In [21]:
# saving the all page content of the whole dataset into a varible
final_data=''
for i in data:
    final_data += i.page_content

In [ ]:
final_data

In [26]:
# importing TokenTextSplitter from langchain.text_splitter. we need it for
# splitting the whole documents because every model has specific token limitations
from langchain.text_splitter import TokenTextSplitter

# now we will call the TokenTextSplitter() class and making an object of it
splitted_docs = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 10000,
    chunk_overlap = 200
)

In [ ]:
# calling the function split_text() through the object we just created
# and passing the final_docs
chunked_docs = splitted_docs.split_text(final_data)
# len(chunked_docs)

In [30]:
# importing Document from langchain.docstore.document
from langchain.docstore.document import Document

In [36]:
# list comprehension
documents = [Document(page_content = t) for t in chunked_docs]
len(documents)

1

In [ ]:
# showing the whole dataset
documents

In [41]:
# creating an obejct of TokenTextSplitter() class
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [42]:
# calling split_documents() function through the object
document_answer_gen = splitter_ans_gen.split_documents(
    documents
)

In [43]:
document_answer_gen
len(document_answer_gen)

4

In [44]:
# importing ChatOpenAI from from langchain.chat_models
from langchain.chat_models import ChatOpenAI

In [46]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
# creating an object of ChatOpenAI() class
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

In [48]:
# prompt designing
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [49]:
# importing PromptTemplate from langchain.prompts
from langchain.prompts import PromptTemplate

In [50]:
# calling the function PromptTemplate() and passing few parameters
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [53]:
# this is the refinig the template

final_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [54]:
# calling PromptTemplate() function and saving into a variable
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [55]:
# importing load_summarize_chain from langchain.chains.summarize
# here load_summarize_chain would be either class or function
# and langchain.chains.summarize is coming from the path langchain/chains/summarize
from langchain.chains.summarize import load_summarize_chain

In [56]:
# calling load_summarize_chain() function
ques_gen_chain = load_summarize_chain(
    llm = llm_ques_gen_pipeline, 
    chain_type = "refine", 
    verbose = True, 
    question_prompt=PROMPT_QUESTIONS, 
    refine_prompt=REFINE_PROMPT_QUESTIONS
    )

In [59]:
# run the pipeline
# ques = ques_gen_chain.run(documents)
# print(ques)